In [149]:
#!pip install --ignore-installed --upgrade tensorflow 
#!pip install opencv-python
#!pip install --upgrade albumentations --user
#!pip install pandas
#!pip install numpy
#!pip install matplotlib

In [150]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19
import albumentations as A # used for image augmentation
from PIL import Image #used for importing images
import re
import os,glob
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , MaxPooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard

In [203]:
tf.config.run_functions_eagerly(True)

In [151]:
data = pd.read_csv('./Genshin_Dataset.csv')
data.head(24)

,Name,Model,Region,Vision,Weapon,Revenue,Year Released
0,Albedo,https://i.postimg.cc/vHqgw9cj/unnamed.jpg,Mondstadt,Geo,Sword,"$11,816,107",2021
1,Ayaka,https://i.postimg.cc/mZP9x58Q/ayaka.jpg,Inazuma,Cryo,Sword,"$16,451,006",2021
2,Ayato,https://i.postimg.cc/3JpDNDMN/ayato.jpg,Inazuma,Hydro,Sword,"$14,481,796",2022
3,Childe (Tartaglia),https://i.postimg.cc/RZBJHV9R/childe.jpg,Liyue,Hydro,Bow,"$13,443,619",2020
4,Cyno,https://i.postimg.cc/bNHN40cx/cyno.jpg,Sumeru,Electro,Polearm,"$13,797,833",2022
5,Eula,https://i.postimg.cc/NMHQDmkr/eula.jpg,Mondstadt,Cryo,Claymore,"$12,619,390",2021
6,Ganyu,https://i.postimg.cc/wM5g8zPq/ganyu.jpg,Liyue,Cryo,Bow,"$15,669,918",2021
7,Hu Tao,https://i.postimg.cc/6pW5K7tD/HuTao.jpg,Liyue,Pyro,Polearm,"$12,481,634",2021
8,Itto,https://i.postimg.cc/NGRgr803/Itto.jpg,Inazuma,Geo,Claymore,"$13,404,072",2022
9,Kazuha,https://i.postimg.cc/NFvBzFdC/kazuha.jpg,Inazuma,Anemo,Sword,"$9,807,112",2021


In [152]:
# given the filepath to the image, we can visualize the image
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [153]:
# transformations using albumentations library
vertical_transform = A.Compose([A.VerticalFlip(p=1)])
horizontal_transform = A.Compose([A.HorizontalFlip(p=1)])


In [154]:
revenues = data['Revenue']
revenues = revenues.map(lambda x: int(re.sub(",", "", x[1:]))) # removes $ and commas from the revenue
data['Revenue (as int)'] = revenues

In [155]:
print("Minimum revenue a character made: ", min(data['Revenue (as int)']))
print("Maximum revenue a character made: ", max(data['Revenue (as int)']))
# using the min max revenue a character made, we divided the data into 2 classifications 
# each classification represents below $15M (0) and above $15M (1)
for row, col in data.iterrows():
    if col['Revenue (as int)'] <= 15000000:
        data.at[row, 'Revenue Classifier'] = 0
    else:
        data.at[row, 'Revenue Classifier'] = 1

data.head()

Minimum revenue a character made:  6825814
Maximum revenue a character made:  33020905


,Name,Model,Region,Vision,Weapon,Revenue,Year Released,Revenue (as int),Revenue Classifier
0,Albedo,https://i.postimg.cc/vHqgw9cj/unnamed.jpg,Mondstadt,Geo,Sword,"$11,816,107",2021,11816107,0.0
1,Ayaka,https://i.postimg.cc/mZP9x58Q/ayaka.jpg,Inazuma,Cryo,Sword,"$16,451,006",2021,16451006,1.0
2,Ayato,https://i.postimg.cc/3JpDNDMN/ayato.jpg,Inazuma,Hydro,Sword,"$14,481,796",2022,14481796,0.0
3,Childe (Tartaglia),https://i.postimg.cc/RZBJHV9R/childe.jpg,Liyue,Hydro,Bow,"$13,443,619",2020,13443619,0.0
4,Cyno,https://i.postimg.cc/bNHN40cx/cyno.jpg,Sumeru,Electro,Polearm,"$13,797,833",2022,13797833,0.0


In [181]:
images_arr = [] # array of images
revenue_arr = [] # array of revenues for each character
counter = 0

# iterates over the character images in the folder 
# returns a tuple where the 3rd element is a list of the character names
for e_image in os.walk('./Genshin Characters/'):

    # iterates over the list of the character names
    for img_name in e_image[2]:
        file_path = './Genshin Characters/{}'.format(img_name)
        image = cv2.imread(file_path, cv2.COLOR_BGR2RGB) # converts the image to color
        image = cv2.resize(image, (200, 200)) # resizing images to same size
        
        # the following code changes pixels to numeric data
        image = np.array(image)
        image = image.astype('float32')
        image /= 255
        
        # transform function takes in the image as the parameter
        # the output is a dictionary with 'image' as the key and the transformed image as the value
        # so, we extract the value by giving it the key
        vertical_image = vertical_transform(image=image)['image'] 
        horizontal_image = horizontal_transform(image=image)['image']
       
        images_arr.append(image)
        images_arr.append(vertical_image)
        images_arr.append(horizontal_image)
        
        revenue_arr.append(data['Revenue Classifier'][counter])
        revenue_arr.append(data['Revenue Classifier'][counter])
        revenue_arr.append(data['Revenue Classifier'][counter])
        counter += 1

        
    



In [182]:
print(len(revenue_arr))
print(len(images_arr))

72
72


In [191]:
images_arr = np.array(images_arr).astype(np.float32)
#images_arr = images_arr.reshape(72, 200*200, 3)
#images_arr = images_arr[:, :, 1] # only green spectrum
#plt.imshow(images_arr[0].reshape(200,200))

In [193]:
revenue_arr = list(map(lambda x: x.astype(np.int32), revenue_arr))

In [194]:
img_train, img_test, rev_train, rev_test = train_test_split(images_arr, revenue_arr, test_size=0.2, shuffle = True, random_state = 20)

In [195]:
img_train.shape

(57, 200, 200, 3)

In [196]:
base_model = keras.applications.Xception (
    weights = 'imagenet',
    input_shape = (200, 200, 3),
    include_top = False
)

In [197]:
base_model.trainable = False

In [198]:
#n_classes = 2
#model = tf.keras.Sequential()
#model.add(tf.keras.layers.Dense(128))
#model.add(tf.keras.layers.ReLU())
#model.add(tf.keras.layers.Dense(128))
#model.add(tf.keras.layers.ReLU())
#model.add(tf.keras.layers.Dense(n_classes))
#model.add(tf.keras.layers.Softmax()) # softmax takes some input _> some probabiltiy distribution

inputs = keras.Input(shape = (200, 200, 3))
x = base_model(inputs, training = False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# you cna use cross entropy as your loss
# your final layer a single output neuron -> that holds what ever the regression prediction is
# for classification your final layer is n_classes of neurons -> where accuracy is argmax(final_layer) is the class


In [199]:
#model.compile(optimizer="adam", loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.compile(optimizer=keras.optimizers.Adam(), loss = keras.losses.BinaryCrossentropy(from_logits=True), metrics = [keras.metrics.BinaryAccuracy()])

In [200]:
print(len(rev_train))
print(len(img_train))

57
57


In [204]:
model.fit(img_train, np.array(rev_train), epochs=20, validation_data = (img_test, np.array(rev_test)))
#model.fit(img_train, np.array(rev_train), epochs=20, validation_data = (img_test, np.array(rev_test)))

Epoch 1/20


C:\Users\wfweg\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2/2 [==============================] - 17s 8s/step - loss: 0.6520 - binary_accuracy: 0.5965 - val_loss: 0.5817 - val_binary_accuracy: 0.8000
Epoch 2/20
2/2 [==============================] - 16s 8s/step - loss: 0.6033 - binary_accuracy: 0.5965 - val_loss: 0.5690 - val_binary_accuracy: 0.8000
Epoch 3/20
2/2 [==============================] - 16s 8s/step - loss: 0.5633 - binary_accuracy: 0.5965 - val_loss: 0.5595 - val_binary_accuracy: 0.8000
Epoch 4/20
2/2 [==============================] - 16s 8s/step - loss: 0.5290 - binary_accuracy: 0.6667 - val_loss: 0.5588 - val_binary_accuracy: 0.8667
Epoch 5/20
2/2 [==============================] - 17s 8s/step - loss: 0.4936 - binary_accuracy: 0.7018 - val_loss: 0.5504 - val_binary_accuracy: 0.8667
Epoch 6/20
2/2 [==============================] - 17s 8s/step - loss: 0.4635 - binary_accuracy: 0.7368 - val_loss: 0.5340 - val_binary_accuracy: 0.8667
Epoch 7/20
2/2 [==============================] - 17s 8s/step - loss: 0.4345 - binary_accuracy: 0.7

In [166]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 128)               5120128   
                                                                 
 re_lu_16 (ReLU)             (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 128)               16512     
                                                                 
 re_lu_17 (ReLU)             (None, 128)               0         
                                                                 
 dense_29 (Dense)            (None, 2)                 258       
                                                                 
 softmax_8 (Softmax)         (None, 2)                 0         
                                                                 
Total params: 5,136,898
Trainable params: 5,136,898
N